In [1]:
# ETL PRoject

In [6]:
import pandas as pd
import psycopg2
import sqlalchemy
import numpy as np
import matplotlib as plt
from sqlalchemy import create_engine, inspect
from datetime import datetime

In [7]:
# Database connection 
pg_port = '5432'
pg_username = 'postgres'
pg_pw = 'postgres'
pg_db = 'movies'
postgresDSN = ('postgresql://{}:{}@{}:{}/{}'.format(pg_username, pg_pw, 'localhost', pg_port, pg_db))
engine = create_engine(postgresDSN)
cnx = engine.connect()

In [ ]:
# create the database schema
# Query was run directly in postgres. Can also be run from jupyter notes

query = f""" 

Drop Table IF EXISTS Movie_Actor CASCADE;  
Drop Table IF EXISTS Ratings CASCADE;  
Drop Table IF EXISTS Movies CASCADE;  
Drop Table IF EXISTS Actors CASCADE;  

CREATE TABLE  Actors  (
     actor_id  INT   NOT NULL,
     first_name  VARCHAR(255)   NOT NULL,
     last_name  VARCHAR(255)   NOT NULL,
     birth_name  VARCHAR(255),
     date_of_birth  text,
     place_of_birth  text,
     date_of_death  text,
     reason_of_death  text,
    CONSTRAINT  pk_Actors  PRIMARY KEY (
         actor_id 
     )
);

CREATE TABLE  Movies  (
     movie_id  INT   NOT NULL,
     title  text   NOT NULL,
     year  INT   NOT NULL,
     genre  VARCHAR(255)   NOT NULL,
     duration  INT,
     language  text,
     director  VARCHAR(255),
     actors  text,
     description  text,
     avg_vote  float   NOT NULL,
     budget  float,
     gross_income  float,
     critic_reviews  float,
    CONSTRAINT  pk_Movies  PRIMARY KEY (
         movie_id 
     )
);

CREATE TABLE  Ratings  (
     movie_id  INT   NOT NULL,
     avg_vote_females  float,
     avg_vote_males  float,
     total_votes  INT   NOT NULL,
     weighted_average_vote  float   NOT NULL,
    CONSTRAINT  pk_Ratings  PRIMARY KEY (
         movie_id 
     )
);

CREATE TABLE  Movie_Actor  (
     id  serial   NOT NULL,
     movie_id  INT   NOT NULL,
     actor_id  INT   NOT NULL,
     role  text,
     characters  text,
    CONSTRAINT  pk_Movie_Actor  PRIMARY KEY (
         id 
     )
);

ALTER TABLE  Ratings  ADD CONSTRAINT  fk_Ratings_movie_id  FOREIGN KEY( movie_id )
REFERENCES  Movies  ( movie_id );

ALTER TABLE  Movie_Actor  ADD CONSTRAINT  fk_Movie_Actor_movie_id  FOREIGN KEY( movie_id )
REFERENCES  Movies  ( movie_id );

ALTER TABLE  Movie_Actor  ADD CONSTRAINT  fk_Movie_Actor_actor_id  FOREIGN KEY( actor_id )
REFERENCES  Actors  ( actor_id );

"""

In [8]:
data_source = 'Resources/'

In [9]:
movies_csv = data_source + 'IMDB movies.csv'
actors_csv = data_source + 'IMDB names.csv'
ratings_csv = data_source + 'IMDB ratings.csv'
title_principals = data_source + 'IMDb title_principals.csv'

In [10]:
actors = pd.read_csv(actors_csv, low_memory=False)

In [11]:
actors.head(3)

,imdb_name_id,name,birth_name,date_of_birth,place_of_birth,date_of_death,reason_of_death
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,1899-05-10,"Omaha, Nebraska, USA",6/22/87,pneumonia
1,nm0000002,Lauren Bacall,Betty Joan Perske,9/16/24,"The Bronx, New York City, New York, USA",8/12/14,stroke
2,nm0000003,Brigitte Bardot,Brigitte Bardot,9/28/34,"Paris, France",NaN,NaN


In [12]:
actors.rename(columns={'imdb_name_id':'actor_id'}, inplace=True)

In [13]:
actors_cols = ['actor_id','name','birth_name','date_of_birth','place_of_birth','date_of_death','reason_of_death']

In [14]:
actors = actors[actors_cols]

In [15]:
actors.head(3)

,actor_id,name,birth_name,date_of_birth,place_of_birth,date_of_death,reason_of_death
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,1899-05-10,"Omaha, Nebraska, USA",6/22/87,pneumonia
1,nm0000002,Lauren Bacall,Betty Joan Perske,9/16/24,"The Bronx, New York City, New York, USA",8/12/14,stroke
2,nm0000003,Brigitte Bardot,Brigitte Bardot,9/28/34,"Paris, France",NaN,NaN


In [16]:
# Cleaning currency column
def clean_currency(x):
    """ If the value is a string with currency symbol, split the string and 
    return numeric value. Else return original input. 
    """
    if isinstance(x, str):
        try: 
            return x.split()[1]
        except IndexError as e:
            return x
    return(x)

In [17]:
# Clean date columns - reove string from text from date 
def clean_date(x):
    for i in str(x).split():
        if i.isdigit():
            return i
        if i.isalpha():
            pass
        else:
            return x

In [18]:
# Check actor id, remover 'nm' prefix and returns numeric portion. Else skips the row if its not in 'nm000123' format
def clean_id(x):
    if x[2:].isdigit() and x[0:2] == 'nm':
        return x[2:]
    else:
        pass

In [19]:
# Clean the data
actors['actor_id'] = actors['actor_id'].apply(lambda x: x.replace('nm', ''))
actors['first_name'] = actors.loc[:,'name'].apply(lambda x: x.split()[0])
actors['last_name'] = actors.loc[:,'name'].apply(lambda x: x.split()[1:])
actors['last_name'] = actors.loc[:,'last_name'].astype(str).apply(lambda x: x.strip('][').replace("'", ""))
actors['date_of_birth'] = actors.loc[:,'date_of_birth'].apply(lambda x: clean_date(x))
actors['date_of_death'] = actors.loc[:,'date_of_death'].apply(lambda x: clean_date(x))
actors = actors[['actor_id','first_name','last_name','birth_name','date_of_birth','place_of_birth','date_of_death','reason_of_death']]

In [20]:
actors.head(3)

,actor_id,first_name,last_name,birth_name,date_of_birth,place_of_birth,date_of_death,reason_of_death
0,0000001,Fred,Astaire,Frederic Austerlitz Jr.,1899-05-10,"Omaha, Nebraska, USA",6/22/87,pneumonia
1,0000002,Lauren,Bacall,Betty Joan Perske,9/16/24,"The Bronx, New York City, New York, USA",8/12/14,stroke
2,0000003,Brigitte,Bardot,Brigitte Bardot,9/28/34,"Paris, France",None,NaN


In [21]:
list(actors.columns)

['actor_id',
 'first_name',
 'last_name',
 'birth_name',
 'date_of_birth',
 'place_of_birth',
 'date_of_death',
 'reason_of_death']

### Movies Data 

In [22]:
movies = pd.read_csv(movies_csv, low_memory=False)

In [23]:
rename_cols = {
    'imdb_title_id':'movie_id',
    'worlwide_gross_income':'gross_income',
    'reviews_from_critics': 'critic_reviews'
}

In [24]:
movies.rename(columns=rename_cols, inplace=True)

In [25]:
movie_cols = ['movie_id','title','year','genre','duration','language','director','actors', \
             'description', 'avg_vote', 'budget','gross_income','critic_reviews']

In [26]:
movies = movies[movie_cols]

In [27]:
movies.head(3)

,movie_id,title,year,genre,duration,language,director,actors,description,avg_vote,budget,gross_income,critic_reviews
0,tt0000009,Miss Jerry,1894,Romance,45,None,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,NaN,NaN,2.0
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,None,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,$ 2250,NaN,7.0
2,tt0001892,Den sorte drøm,1911,Drama,53,NaN,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,NaN,NaN,2.0


In [28]:
movies['movie_id'] = movies['movie_id'].apply(lambda x: x.replace('tt', ''))
movies['gross_income'] = movies.loc[:,'gross_income'].apply(lambda x: clean_currency(x))
movies['budget'] = movies.loc[:,'budget'].apply(lambda x: clean_currency(x))
movies['year'] = movies.loc[:,'year'].apply(lambda x: clean_date(x))

In [29]:
movies.head(3)

,movie_id,title,year,genre,duration,language,director,actors,description,avg_vote,budget,gross_income,critic_reviews
0,0000009,Miss Jerry,1894,Romance,45,None,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,NaN,NaN,2.0
1,0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,None,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,2250,NaN,7.0
2,0001892,Den sorte drøm,1911,Drama,53,NaN,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,NaN,NaN,2.0


In [30]:
list(movies.columns)

['movie_id',
 'title',
 'year',
 'genre',
 'duration',
 'language',
 'director',
 'actors',
 'description',
 'avg_vote',
 'budget',
 'gross_income',
 'critic_reviews']

### Ratings data

In [31]:
ratings = pd.read_csv(ratings_csv)

In [32]:
rename_cols = {
    'imdb_title_id':'movie_id',
    'females_allages_avg_vote':'avg_vote_females',
    'males_allages_avg_vote':'avg_vote_males'
}

In [33]:
ratings.rename(columns=rename_cols, inplace=True)

In [34]:
ratings_cols = ['movie_id','avg_vote_females','avg_vote_males','total_votes','weighted_average_vote']

In [35]:
ratings = ratings[ratings_cols]

In [36]:
ratings['movie_id'] = ratings['movie_id'].apply(lambda x: x.replace('tt', ''))

In [37]:
ratings.head(3)

,movie_id,avg_vote_females,avg_vote_males,total_votes,weighted_average_vote
0,0000009,6.0,6.2,154,5.9
1,0000574,6.2,6.1,589,6.1
2,0001892,5.7,5.9,188,5.8


In [38]:
list(ratings.columns)

['movie_id',
 'avg_vote_females',
 'avg_vote_males',
 'total_votes',
 'weighted_average_vote']

### movie_actors table

In [39]:
movie_actor = pd.read_csv(title_principals)

In [40]:
movie_actor.head(3)

,imdb_title_id,ordering,imdb_name_id,category,job,characters
0,tt0000009,1,nm0063086,actress,NaN,"[""Miss Geraldine Holbrook (Miss Jerry)""]"
1,tt0000009,2,nm0183823,actor,NaN,"[""Mr. Hamilton""]"
2,tt0000009,3,nm1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor..."


In [41]:
rename_cols = {
    'imdb_title_id':'movie_id',
    'imdb_name_id':'actor_id',
    'category': 'role'
}

In [42]:
movie_actor.rename(columns=rename_cols, inplace=True)

In [43]:
movie_actor['movie_id'] = movie_actor['movie_id'].apply(lambda x: x.replace('tt', ''))
movie_actor['actor_id'] = movie_actor['actor_id'].apply(lambda x: x.replace('nm', ''))

In [45]:
movie_actor.head(3)

,movie_id,ordering,actor_id,role,job,characters
0,0000009,1,0063086,actress,NaN,"[""Miss Geraldine Holbrook (Miss Jerry)""]"
1,0000009,2,0183823,actor,NaN,"[""Mr. Hamilton""]"
2,0000009,3,1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor..."


In [46]:
movie_actor = movie_actor[['movie_id','actor_id','role','characters']]

In [47]:
list(movie_actor.columns)

['movie_id', 'actor_id', 'role', 'characters']

### Load Sql Tables

In [46]:
# read the data to tables:
movies.to_sql('movies', schema='public', con=cnx, if_exists='append', index=False)

In [150]:
actors.to_sql('actors', schema='public', con=cnx, if_exists='append', index=False)

In [152]:
ratings.to_sql('ratings', schema='public', con=cnx, if_exists='append', index=False)

In [48]:
movie_actor.to_sql('movie_actor', schema='public', con=cnx, if_exists='append', index=False)